In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
import datetime
import random
from neo4j import GraphDatabase



# Fake data creation

In [2]:
from numpy import integer

Faker.seed(1000)
fake=Faker()

money_transaction = defaultdict(list)
account_info=defaultdict(list)

row_number=100



In [3]:
# Creating initiated_account_number as much as the given input
bank_info={"BankAnk":"100","BankIST":"200","BankBUR":"300"}

for i in range(row_number):
    money_transaction['initiated_account_number'].append(random.choice(list(bank_info.values()))+fake.bothify(text='####'))

initiated_account_number=money_transaction["initiated_account_number"]

#Creating other futures
amount=[]
account_number=[]
timestamp=[]

In [4]:
for idx in range(len(money_transaction['initiated_account_number'])): #For loop for creating random values in features
    temp_account_id_list=[]
    timestamp_list=[]
    amount_list=[]
    a=random.randint(1,9)
    #Creating random relationship on two nodes
    for j in range(a): 

        temp_amount_list=[]
        temp_timestamp_list=[]

       
        #Choosing random initiated_account_number and adding to reciever_id
        rnd_account_id=''.join(random.choices(initiated_account_number)) 

        if (initiated_account_number[idx]== rnd_account_id):
            continue
        for i in range(random.randint(1,a)):
            temp_amount_list.append(fake.pyfloat(right_digits=2,positive=True,min_value=2.00,max_value=10000.00))
            temp_timestamp_list.append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))
            
        amount_list.append(temp_amount_list)
        temp_account_id_list.append(rnd_account_id)
        timestamp_list.append(temp_timestamp_list)    


    #Appending every feature value to list
    

    #Check the size of the receiver_id compare it with initiated_account_number
    if len(account_number)==len(money_transaction['initiated_account_number']):
            break


    amount.append(amount_list)
    account_number.append(temp_account_id_list)
    timestamp.append(timestamp_list)
        

In [5]:
#Ejecting fake/manipulated data do Dataframe
money_transaction['amount']=amount
money_transaction['account_number']=account_number
money_transaction['timestamp']=timestamp
print(len(money_transaction['amount']))

100


In [6]:

df=pd.DataFrame(money_transaction)
df

,initiated_account_number,amount,account_number,timestamp
0,3006165,[[1600.81]],[1004241],[[2022-10-16 17:50:36]]
1,3001728,"[[3200.73], [3734.98, 2885.61], [9455.42, 4821...","[1003359, 1007579, 3001532, 1009032, 1006976, ...","[[2022-10-17 06:36:50], [2022-11-01 02:36:52, ..."
2,3006233,"[[4470.93, 2771.72, 1167.28, 6641.64, 5084.16,...","[2003039, 3006155, 3008122, 2004385, 1002582, ...","[[2022-09-30 23:58:49, 2022-10-04 03:32:56, 20..."
3,1005735,"[[6808.62], [7864.47, 5199.25, 1936.88], [3652...","[1006804, 1003424, 1007579]","[[2022-11-03 03:07:21], [2022-11-05 14:38:00, ..."
4,1003720,"[[7245.77], [365.13, 971.29], [8348.41, 9158.72]]","[1005735, 3004939, 3006760]","[[2022-09-16 11:37:49], [2022-10-16 06:59:25, ..."
...,...,...,...,...
95,1002955,"[[825.23, 2148.47, 3218.85], [747.81, 333.23, ...","[2004385, 1003465, 2002155, 3006165]","[[2022-09-16 03:05:54, 2022-10-07 21:53:48, 20..."
96,2009839,"[[778.62, 5214.73, 4334.95], [3593.45], [5680....","[2000956, 3001532, 3000614, 2000504, 3003756]","[[2022-11-03 10:53:35, 2022-10-10 04:32:15, 20..."
97,1008492,"[[5942.91, 8686.37, 9125.42, 3692.71], [7129.6...","[1002516, 1007653, 1003424, 1004819]","[[2022-11-09 00:08:29, 2022-10-15 05:38:59, 20..."
98,2009956,"[[2733.5, 9817.28], [7549.48, 2930.28, 6036.84...","[1008778, 1005554, 3001302]","[[2022-09-19 13:27:02, 2022-10-18 20:42:13], [..."


In [21]:
account_info['account_id']=[]
account_info['account_number']=[]
account_info['name']=[]
account_info['phone']=[]
account_info['address']=[]

for i in range(row_number):
    account_info['account_id'].append(1001+i)
    account_info['name'].append(fake.name())

    account_info['phone'].append(fake.phone_number())
    account_info['address'].append(fake.address())

account_info['account_number']=df['initiated_account_number']

In [22]:
account_df=pd.DataFrame(account_info)
account_df

,account_id,name,phone,address,account_number
0,1001,Sherri Harris,1259159854,"8433 Daniel Ways\nWilliamsberg, CO 57908",3006165
1,1002,Gabriela Martin,001-606-309-8181,"3370 Neal Loaf Suite 113\nReedchester, AR 56686",3001728
2,1003,Brian Oconnor,878-125-4689x5619,"0091 Roberts Grove\nPaulhaven, ND 56589",3006233
3,1004,Wesley Drake,6435587527,"74144 Erica Streets Suite 490\nNew Janice, KY ...",1005735
4,1005,Wendy Stone,+1-745-913-1204x756,"1124 Lori Parkway\nEast Angelamouth, AZ 44680",1003720
...,...,...,...,...,...
95,1096,Beth Boone,001-380-556-9204x060,"01487 Erin Coves Apt. 510\nWest Chelsea, IL 10482",1002955
96,1097,Andrea Rose,350.478.0538,"49916 Dennis Pass Suite 757\nWashingtonfurt, C...",2009839
97,1098,Kyle Sharp,+1-212-478-4942x2784,"61895 Moss Extensions\nErinshire, VA 31549",1008492
98,1099,Judith Thornton,+1-185-449-8978x75711,"1549 Victoria Squares\nPeterton, LA 88975",2009956


# Neo4j

In [23]:
from neo4j import GraphDatabase

In [24]:
transaction_info=df.values.tolist()
account_info=account_df.values.tolist()
transaction_info[2][1][0]

[4470.93, 2771.72, 1167.28, 6641.64, 5084.16, 9250.34]

In [25]:
account_info[0][1]

'Sherri Harris'

In [33]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "test"))

#Deleting existing values and relationships
def clear_db(tx):
    tx.run("MATCH(n) Detach Delete n")

def create_account(tx,account_id,name,phone,address,account_number):
    tx.run("Create(ah:AccountHolder {"
            "account_id: $account_id,"
            "name:$name,"
            "phone:$phone,"
            "address:$address,"
            "account_number:$account_number})",
            account_id=account_id,name=name,phone=phone,address=address,account_number=account_number
    )
#Creating customer node
def create_transaction(tx,initiated_account_number,amount,account_number,timestamp):
    tx.run(
        "Match(ah:AccountHolder{account_number: $initiated_account_number})"
        "Match(ah2:AccountHolder{account_number: $account_number})"
        "Create(mt:MoneyTransaction{"
        "amount: $amount,"
        "timestamp: $timestamp}) "
        "Create(ah)-[:Send]->(mt)-[:Send]->(ah2)",
        initiated_account_number=initiated_account_number,amount=amount,account_number=account_number,timestamp=timestamp)


with driver.session() as session:

    session.execute_write(clear_db)
    for i in range(len(account_info)):
        session.execute_write(create_account,account_info[i][0],account_info[i][1],account_info[i][2],account_info[i][3],account_info[i][4])


    for i in range(len(transaction_info)):
        for j in range(len(transaction_info[i][1])):
            for z in (transaction_info[i][1][j]):
                for k in range(len(transaction_info[i][3][j])):
                   session.execute_write(create_transaction,transaction_info[i][0],z,transaction_info[i][2][j],transaction_info[i][3][j][k])
driver.close()